# 실습5: 네이버 역대 박스오피스 순위, 평점 긁기

1.   역대 박스오피스 상위 100개 영화 목록 크롤링 (100개 긁을 때 영화 하나가 이상한데 그거 잘 처리하기)
2.   평론가평점, 네티즌평점, 관람객 평점 공감순 상위 50개 댓글 평점 평균 수집
3.   평론가평점과 관람객 평점 공감순 상위 50개 댓글 평점 평균이 같은 영화 찾기

#### 참고: https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=%EC%97%AD%EB%8C%80+%EB%B0%95%EC%8A%A4%EC%98%A4%ED%94%BC%EC%8A%A4%EC%88%9C%EC%9C%84

In [1]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd

In [8]:
url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=%EC%97%AD%EB%8C%80+%EB%B0%95%EC%8A%A4%EC%98%A4%ED%94%BC%EC%8A%A4%EC%88%9C%EC%9C%84'

res = req.get(url)
soup = bs(res.text, 'html.parser')

# 만약 영화 리스트를 긁고싶다면?
# 영화 하나를 둘러싸고 있는 html 태그가 있을 것이다.
# 그리고 대부분 a라는 영화를 둘러싸고 있는 태그는 b라는 영화를 둘러싸고 있는 태그와 동일하다.
# 바로 이 점을 주목하여 html 태그를 찾는다.

# 즉, 영화를 둘러싸고 있는, 규칙적인, 반복적으로 나타나는 태그를 찾는 것이다.
# 그 결과 _content라는 class를 가진 div 밑에 ul 태그 밑에 li 태그가 영화 정보를 담고 있으며, 규칙적으로 반복된다는 것을 발견했다.
movierank_li = soup.select('._content > ul > li')

In [77]:
# 데이터를 담아줄 빈 리스트를 선언한다.
# 왜 이런 생각을 하게 됐을까?
# 이전 실습파일에서 주석으로 언급했듯이...

# 크롤링에서 가장 중요한 것은 데이터를 어떻게 구성할 지 미리 그려보는 것이다.
# 영화와 관련된 데이터를 긁을테니 컬럼으로 영화제목, 개봉일, url 등등이 들어갈 것이다.

# 그러면 무슨 기준으로 컬럼을 선정하는가?
# -> 역대박스오피스 순위 페이지에서 규칙적으로 반복되는 영화 정보들을 컬럼으로 선정한다. (ex. 영화제목, 개봉일 등등)

movie_title = []
movie_date = []
movie_ticketsales = []
movie_url = []

In [2]:
# 평점을 긁기 위해서는 영화 상세정보 페이지로 넘어가야 한다.
# 예고편을 누르면 영화 상세정보 페이지로 넘어갈 수 있다.
# 그런데 왜 다운로드를 눌렀을 때 연결되는 url을 긁으려고 하는 것일까?

# 우리가 긁어햐 하는 영화 자료는 크게 두 가지로 나뉜다. 
# (1) 영화제목, 개봉일, 누적관객수.
# (2) 네티즌 평점, 전문가 평점, 공감 순위 등등

# 무슨 기준으로 나뉜 것일까?
# 웹 페이지 단위이다. 즉, (1)을 긁을 수 있는 웹페이지와 (2)를 긁을 수 있는 웹페이지가 다르다.
# 영화 하나에 대한 모든 정보를 긁어오기 위해서 우리는 (1)을 먼저 다 긁고 다른 웹페이지로 접근해서 (2)를 긁어야 한다.
# 그리고 (2)를 긁을 수 있는 웹페이지에 대한 url 정보는 (1)에 담겨있다.

# 각 영화의 상세정보페이지 url은 뒷부분에 영화코드(mcode)만 다를 뿐 나머지는 동일하다.
# 또한, mcode는 다운로드를 눌렀을 때 연결되는 url에 담겨있다.

# 따라서, 우리는 영화 상세정보페이지 url에서 변하지 않는 부분은 고정시켜두고
# mcode만 바꿔주면서 영화 상세정보를 긁고자 한다.
# 그렇기 때문에 다운로드 url을 긁는 것이다.

In [28]:
# 제목
movierank_li[0].select('.movie_tit strong')[0].text

'명량'

In [32]:
# 개봉일
movierank_li[0].select('.movie_item dd')[0].text

' 2014.07.30. '

In [33]:
# 관객수
movierank_li[0].select('.movie_item dd')[1].text

'17,615,844명'

In [48]:
# 영화코드
movierank_li[0].select('.btn_area > a')[0].attrs['href']

'https://serieson.naver.com/tvstore/detail.nhn?mcode=93756'

In [54]:
movierank_li[0].select('.btn_area > a')[0].text

'다운로드'

In [78]:
# 모든 준비가 끝났다.
# 이제 매 반복마다 아까 만들어둔 빈 리스트에 데이터를 append해준다.
# 그런데 중간에 어떤 영화는 예고편이 없으며, 또 어떤 영화는 다운로드 버튼이 없다.
# 이 문제는 if..else..로 처리해주었다.
# 어떻게 해결할 지에 대해서는 정답이 있는 것은 아니다. 다양한 방법으로 시도해보자.

for movie in movierank_li:
    movie_title.append(movie.select('.movie_tit strong')[0].text)
    movie_date.append(movie.select('.movie_item dd')[0].text)
    movie_ticketsales.append(movie.select('.movie_item dd')[1].text)
    
    tmp = movie.select('.btn_area > a')
    
    if len(tmp) == 2:
        if tmp[0].text == '다운로드':
            movie_url.append(tmp[0].attrs['href'])
        else:
            movie_url.append(tmp[1].attrs['href'])
    else:
        if tmp[0].text == '예고편':
            movie_url.append(tmp[0].attrs['href'].split('&')[0])
        else:
            movie_url.append(tmp[0].attrs['href'])

movie_dict = {
    'movie_title':movie_title,
    'movie_date':movie_date,
    'movie_ticketsales':movie_ticketsales,
    'movie_url' : movie_url
}

In [79]:
# (1) 영화제목, 개봉일, 누적관객수
# (1)에 해당하는 정보를 모두 수집했다.
# 앞으로 하는 작업은 (2)를 긁기 위해 movie_url에서 mcode를 따로 추출하는 작업이다.
# mcode를 추출하는 작업 또한 여러가지 방법이 있을 수 있다.
# 다양한 방법으로 시도해보자.

tmp_df = pd.DataFrame(movie_dict)
tmp_df.head()

,movie_title,movie_date,movie_ticketsales,movie_url
0,명량,2014.07.30.,"17,615,844명",https://serieson.naver.com/tvstore/detail.nhn?...
1,극한직업,2019.01.23.,"16,266,338명",https://serieson.naver.com/tvstore/detail.nhn?...
2,신과함께-죄와 벌,2017.12.20.,"14,414,658명",https://serieson.naver.com/tvstore/detail.nhn?...
3,국제시장,2014.12.17.,"14,264,059명",https://serieson.naver.com/tvstore/detail.nhn?...
4,어벤져스: 엔드게임,2019.04.24.,"13,977,602명",https://serieson.naver.com/tvstore/detail.nhn?...


In [63]:
tmp_df['movie_url'][0].split('?')[1]

'mcode=93756'

In [80]:
tmp_df['movie_url'] = tmp_df['movie_url'].apply(lambda x: x.split('?')[1])

In [81]:
tmp_df['movie_url'][41]

'code=39569'

In [82]:
tmp_df['movie_code'] = tmp_df['movie_url'].apply(lambda x: x.split('=')[1])

In [83]:
tmp_df['movie_code']

0      93756
1     167651
2      85579
3     102875
4     136900
       ...  
95    163533
96     52548
97    137326
98     49727
99    144314
Name: movie_code, Length: 100, dtype: object

In [84]:
# mcode를 추출했으면 이제 영화상세정보 페이지를 긁을 차례다.
# 모든 영화들의 상세정보 페이지 url은 mcode만 다를 뿐 아래 target_url은 똑같다.
# 조금 더 구체적으로 말하자면 target_url은 공통사항이며, mcode를 parameter로 받는다. 
code_list = tmp_df['movie_code']

target_url = 'https://movie.naver.com/movie/bi/mi/point.naver?'

# 예를 들면, mcode를 담고있는 code_list의 0번째 인덱스에 해당하는 영화의 상세정보 요청 url은 아래와 같다.
# 'https://movie.naver.com/movie/bi/mi/point.naver?mcode=xxxx'
# params 옵션을 사용해서 요청을 보내자.
res = req.get(target_url, params={'code':code_list[0]})
res


<Response [200]>

In [142]:
# 아래 과정은 코드를 일반화하여 한꺼번에 데이터를 가져오기 전에
# 우리에게 필요한 데이터를 어떻게 가져오는지 test해보는 것이다.
soup = bs(res.text, 'html.parser')
tmp_spc_score = soup.select('.spc_score_area')[0].select('div > em')

# tmp_spc_score를 출력해보면 리스트 안에 숫자가 인덱스로 들어가있을 것이다. (['6', '.' , '2', '9'])
# 실제 우리에게 필요한 정보는 하나의 소숫점으로 이어진 6.29이다. 따라서 join()함수로 붙여준다.
spc_score = ''.join([i.text for i in tmp_spc_score])
spc_score

'6.29'

In [165]:
# 네티즌 평점도 어떻게 갖고 오는지 확인해보자.
tmp_ntz_score = soup.select('.score_left > .star_score em')
ntz_score = ''.join(i.text for i in tmp_ntz_score)
ntz_score

'8.44'

In [202]:
# 이제 우리는 관람객 평점을 공감순으로 나열한 후 상위 50개 평균을 구해야 한다.
# score_result class에서 ul > li로 평점 정보를 가져올 수 있지만, 또 다른 방법이 있다.

# 개발자도구에서 우리가 긁어야할 댓글부분을 확인해보면 아래와 같은 태그를 찾을 수 있다.
# <iframe src="/movie/bi/mi/pointWriteFormList.naver?code=93756&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false">
# iframe은 웹사이트 안에 있는 자그만한 웹사이트라고 생각하면 편하다.
# 관람객 댓글 정보는 사실 하나의 웹사이트로 구성되어있다. 당연히 요청 url도 찾을 수 있다.

# iframe src에 들어가서 개발자도구의 네트워크탭을 클릭해보면 요청 url을 얻을 수 있으며, 파라미터에 따라 여러 옵션을 지정해줄 수 있다. 
# 우리는 나열 순서(order)를 공감순으로 할 것이다. (sympathyScore)

# 아래는 iframe의 url로 요청을 보내는 코드이다.
target_url2 = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?'

param_dict = {
    'code':'93756',
    'type':'after',
    'onlyActualPointYn':'Y',
    'onlySpoilerPointYn':'N',
    'order':'sympathyScore',
    'page':1
}

res2 = req.get(target_url2, params=param_dict)
soup2 = bs(res2.text, 'html.parser')

In [209]:
tmp_act_score = soup2.select('.score_result .star_score em')
act_score = list(map(int, [i.text for i in tmp_act_score]))
sum(act_score)

97

In [210]:
# 50개 평균
# 한 페이지에 10개의 댓글이 들어있다.
# 따라서 우리는 1~5페이지를 긁으면 된다.
# 5번을 반복하면서 관람객 평점 50개를 긁고 평균을 구해준다.
target_url2 = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?'
avg_act_score = 0

for i in range(1,5):
    
    param_dict = {
        'code':'93756',
        'type':'after',
        'onlyActualPointYn':'Y',
        'onlySpoilerPointYn':'N',
        'order':'sympathyScore',
        'page':i
    }

    res2 = req.get(target_url2, params=param_dict)
    soup2 = bs(res2.text, 'html.parser')
    tmp_act_score = soup2.select('.score_result .star_score em')
    act_score = list(map(int, [i.text for i in tmp_act_score]))
    
    avg_act_score += sum(act_score)
    
avg_act_score /= 50
avg_act_score

7.14

In [216]:
# 전체 평점 긁기
# 아래는 위에서 하나씩 테스트해본 결과를 토대로 영화 100개에 대한 평점 정보를 수집하는 코드이다.
# 일반화하여 한번에 모든 영화의 평점 정보를 긁어오는 것이다.
target_url = 'https://movie.naver.com/movie/bi/mi/point.naver?'
target_url2 = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?'

actual_valence = []
netizen_valence = []
critic_valence = []

for movie_code in code_list:
    
    res = req.get(target_url, params={'code':movie_code})
    soup = bs(res.text, 'html.parser')
    
    # 전문가 평점
    try:
        tmp_spc_score = soup.select('.spc_score_area')[0].select('div > em')
        spc_score = ''.join([i.text for i in tmp_spc_score])
        critic_valence.append(float(spc_score))
    except:
        critic_valence.append(0)
    
    # 네티즌 평점
    try:
        tmp_ntz_score = soup.select('.score_left > .star_score em')
        ntz_score = ''.join(i.text for i in tmp_ntz_score)
        netizen_valence.append(float(ntz_score))
    except:
        netizen_valence.append(0)
    
    # 관람객 공감순 50개 평균 평점
    avg_act_score = 0

    for j in range(1,5):

        param_dict = {
            'code':movie_code,
            'type':'after',
            'onlyActualPointYn':'Y',
            'onlySpoilerPointYn':'N',
            'order':'sympathyScore',
            'page':j
        }

        res2 = req.get(target_url2, params=param_dict)
        soup2 = bs(res2.text, 'html.parser')
        tmp_act_score = soup2.select('.score_result .star_score em')
        act_score = list(map(int, [i.text for i in tmp_act_score]))

        avg_act_score += sum(act_score)

    avg_act_score /= 50
    actual_valence.append(avg_act_score)

In [218]:
movie_dict

{'movie_title': ['명량',
  '극한직업',
  '신과함께-죄와 벌',
  '국제시장',
  '어벤져스: 엔드게임',
  '겨울왕국 2',
  '베테랑',
  '아바타',
  '도둑들',
  '7번방의 선물',
  '알라딘',
  '암살',
  '범죄도시2',
  '광해, 왕이 된 남자',
  '신과함께-인과 연',
  '택시운전사',
  '부산행',
  '변호인',
  '해운대',
  '어벤져스: 인피니티 워',
  '실미도',
  '괴물',
  '왕의 남자',
  '어벤져스: 에이지 오브 울트론',
  '인터스텔라',
  '기생충',
  '겨울왕국',
  '보헤미안 랩소디',
  '검사외전',
  '엑시트',
  '설국열차',
  '관상',
  '아이언맨 3',
  '캡틴 아메리카: 시빌 워',
  '해적: 바다로 간 산적',
  '수상한 그녀',
  '국가대표',
  '백두산',
  '과속스캔들',
  '탑건: 매버릭',
  '스파이더맨: 파 프롬 홈',
  '디 워',
  '공조',
  '트랜스포머 3',
  '히말라야',
  '스파이더맨: 노 웨이 홈',
  '미션 임파서블 : 고스트 프로토콜',
  '밀정',
  '최종병기 활',
  '써니',
  '트랜스포머',
  '트랜스포머: 패자의 역습',
  '스파이더맨: 홈커밍',
  '한산: 용의 출현',
  '1987',
  '베를린',
  '마스터',
  '터널',
  '어벤져스',
  '내부자들',
  '늑대소년',
  '인천상륙작전',
  '럭키',
  '은밀하게 위대하게',
  '범죄도시',
  '곡성(哭聲)',
  '화려한 휴가',
  '좋은 놈, 나쁜 놈, 이상한 놈',
  '공조2: 인터내셔날',
  '군함도',
  '미션 임파서블: 폴아웃',
  '웰컴 투 동막골',
  '다크 나이트 라이즈',
  '사도',
  '아저씨',
  '킹스맨 : 시크릿 에이전트',
  '미션 임파서블: 로그네이션',
  '미녀는 괴로워',
  '전우치',
  '연평해전',
  '인셉션',
  '

In [219]:
# 새롭게 수집한 평점 정보도 기존 딕셔너리에 추가해준다.
movie_dict['critic_valence'] = critic_valence
movie_dict['netizen_valence'] = netizen_valence
movie_dict['actual_valence'] = actual_valence

In [220]:
df = pd.DataFrame(movie_dict)
df.head()

,movie_title,movie_date,movie_ticketsales,movie_url,critic_valence,netizen_valence,actual_valence
0,명량,2014.07.30.,"17,615,844명",https://serieson.naver.com/tvstore/detail.nhn?...,6.29,8.44,7.14
1,극한직업,2019.01.23.,"16,266,338명",https://serieson.naver.com/tvstore/detail.nhn?...,6.80,8.51,7.30
2,신과함께-죄와 벌,2017.12.20.,"14,414,658명",https://serieson.naver.com/tvstore/detail.nhn?...,5.92,7.83,4.28
3,국제시장,2014.12.17.,"14,264,059명",https://serieson.naver.com/tvstore/detail.nhn?...,5.81,9.02,7.72
4,어벤져스: 엔드게임,2019.04.24.,"13,977,602명",https://serieson.naver.com/tvstore/detail.nhn?...,7.62,9.38,7.92


In [222]:
df['critic_actual_diff'] = abs(df['actual_valence'] - df['critic_valence'])

In [225]:
# '미션 임파서블: 로그네이션'이 평론가 평점과 관람객 공감순 상위 50개 평점 평균이 같은 영화로 나왔다.
# 만약 결과가 다르다면 iframe의 요청 url 파라미터를 확인해보자.
df = df.sort_values('critic_actual_diff')
df.reset_index(inplace=True, drop=True)
df.loc[(df['critic_valence'] != 0) & (df['actual_valence'] != 0) & (df['critic_actual_diff']==0)]

,index,movie_title,movie_date,movie_ticketsales,movie_url,critic_valence,netizen_valence,actual_valence,critic_netizen_diff,critic_actual_diff
3,76,미션 임파서블: 로그네이션,2015.07.30.,"6,126,488명",https://serieson.naver.com/tvstore/detail.nhn?...,7.54,8.75,7.54,0.0,0.0


# 수고하셨습니다!!